In [15]:
##### import tempfile
import numpy as np
import pandas as pd
#import scipy.optimize as spo
import matplotlib.pyplot as plt
from matplotlib import style

import collections
import headers as HEADER
import os, stat, os.path, time, datetime, subprocess
import re, sys, glob, argparse, csv, shutil, fileinput
from pprint import pprint
from itertools import islice
import operator
import ConfigParser
from collections import defaultdict



from jinja2 import Environment, FileSystemLoader

style.use('ggplot')


def parse_args():
    parser = argparse.ArgumentParser(
        description='get coverage for a list of positions')
    parser.add_argument(
        '-m', '--meta_file',
        help='specify input file which is meta file',
        required=True)
    parser.add_argument(
        '-b', '--bed_regions',
        help='specify region of interest in bed format',
        required=True)
    args = parser.parse_args()
    return args


def get_files(bam_vcf_files, input_headers):
    """Dictionary:{patient->{status->{file_identifier -> file_path}}}"""
    patient_files = dict()
    with open(bam_vcf_files, 'r') as fh:
        records = csv.DictReader(fh,  delimiter='\t')
        headers = set(records.fieldnames)
        input_headers = set(input_headers)
        # check if all mandatory columns prsent in input file
        if input_headers.issubset(headers):
            print "input file has all mandatory columns! Continue..."
            for line in records:
                patient = line['patient']
                status = line['status']
                if patient not in patient_files:
                    patient_files[patient] = {}
                if status not in patient_files[patient]:
                    patient_files[patient][status] = line
                else:
                    print 'Duplicate status entries! One tissue sequenced multiple times?'
                    sys.exit()
        else:
            print "Input file doesn't contain all mandatory headers!\n %s" % list(input_headers)
            sys.exit()
        return patient_files
    
    

def make_BEDcoverage_script(patient_files, bed_regions, template_dir):
    bed_path = "/home/rcorbett/aligners/bedtools/BEDTools-Version-2.15.0/bin/"
    sam_path = "/home/rcorbett/aligners/samtools/samtools-0.1.17/"
    wkdir = os.getcwd()
    scripts = []
    stamps = []
    coverage_files = []
    for patient in patient_files:
        for status in patient_files[patient]:
            library = patient_files[patient][status]['DNA_lib']
            bam = patient_files[patient][status]['DNA_bam'] 
            complete_stamp = ".".join([patient, status, library, "complete"])
            stamps.append(complete_stamp)
            coverage_file = ".".join([patient, status, library, "exon.coverage.txt"])
            coverage_files.append(coverage_file)
            script = populate_template(patient,
                                       status,
                                       wkdir,
                                       library,
                                       bed_regions,
                                       bam,
                                       template_dir)
            scripts.append(script)
    return [scripts, stamps, coverage_files]
    
    
def populate_template(patient, status, wkdir, library,
                      bed_regions, bam, template_dir):
    bedcoverage_script = ".".join([patient, status, library, "BEDcoverage.sh"])
    jinja2_env = Environment(loader=FileSystemLoader([template_dir]),
                             trim_blocks=True)
    template = jinja2_env.get_template('bedcoverage_template.sh')
    with open(bedcoverage_script, 'wb') as opf:
        content = template.render(patient=patient,
                                  status=status,
                                  library=library,
                                  bam=bam,
                                  bed_regions=bed_regions,
                                  wkdir=wkdir)
        opf.write(content)
#         print 'templated {0}'.format(bedcoverage_script)
    return bedcoverage_script


# def make_dir(path):
#     if not os.path.exists(path):
#         os.makedirs(path)
#     else:
#         print "%s directory already exists!" % path
      


def intervals_to_dict(gene_intervals_file):
    """Dictionary holds {gene-> [chromosome, start, end]}"""
    with open(gene_intervals_file, 'r') as fh:
        records = csv.DictReader(fh,  delimiter='\t')
        gene_regions = dict()
        for line in records:
            gene = line['gene'].split('_')[0]
            chromosome = line['chr']
            start = line['start']
            end = line['end']
            info = [chromosome, start, end]
            try:
                gene_regions[gene].append(info)
            except KeyError:
                gene_regions[gene] = [info]
    return gene_regions
            
           
def  intervals_to_regions(gene_regions):
    """Calculate bedcoverage region for a gene including 1 kb up and downstream"""
    for gene in  gene_regions:
        #make_dir(gene)
        #region_file = os.getcwd() + "/" + gene + "/" + gene + "_combined_exon_pm_1kb.txt"
        region_file = os.getcwd() + "/" + gene + "_combined_exon_pm_1kb.txt"
        starts = [int(i[1]) for i in gene_regions[gene]]
        ends = [int(i[2]) for i in gene_regions[gene]]
        chromosome = gene_regions[gene][0][0]
        # expand region to 1 kb up and downstream the gene
        interested_start = np.array(starts).min() - 1000
        interested_end = np.array(ends).max() + 1000
        with open(region_file, 'w') as opf:
            # csv writer always generate a dos file
            writer = csv.writer(opf,  delimiter='\t', lineterminator="\n")
            writer.writerow([chromosome, interested_start, interested_end, gene])
            
            
def qsub_scripts(scripts, wkdir):
    """qsub scripts"""
    for script in scripts:
        print "cd to %s" % wkdir
        print "submit %s" % script
        p = subprocess.Popen('ssh tachpc \"cd %s;  qsub %s\"' %
                             (wkdir, script),  shell=True,
                             stdout=subprocess.PIPE)
        output,  err = p.communicate()

        
# def make_submit_bedscripts_all_genes(patient_files):
#     os.chdir('/home/szong/projects/development/coverage')
#     wkdir = os.getcwd()
#     #for gene in gene_regions:
#     #new_dir =  wkdir + "/" + gene
#     #os.chdir(new_dir)
#     #region_file = gene + "_combined_exon_pm_1kb.txt"
#     region_file = "all_genes_combined_exons.txt"
#     out = make_BEDcoverage_script(patient_files, region_file, template_dir)
#     BEDcoverage_scripts = out[0]
#     complete_stamps = out[1]
#     coverage_files = out[2]
# #         qsub_scripts(BEDcoverage_scripts, new_dir)
# #     qsub_scripts(BEDcoverage_scripts, wkdir)

In [20]:
os.chdir('/projects/trans_scratch/validations/workspace/szong/AML_capture/coverage/bedcoverage')


In [16]:
start = datetime.datetime.now()
meta_file = "/home/szong/projects/development/coverage/vcf_bam_all_samples_new_20150924.csv"
gene_intervals_file = "/home/szong/projects/development/coverage/VisCapCancer_input_interval_file.bed"
template_dir = '/home/szong/projects/development/coverage/template/'
input_headers = HEADER.INPUT_FILE_HEADER
patient_files = get_files(meta_file, input_headers)
region_file = "/home/szong/projects/development/coverage/all_genes_combined_exons.txt"
out = make_BEDcoverage_script(patient_files, region_file, template_dir)
BEDcoverage_scripts = out[0]
complete_stamps = out[1]
coverage_files = out[2]

os.chdir('/projects/trans_scratch/validations/workspace/szong/AML_capture/coverage/bedcoverage')
wkdir = os.getcwd()

qsub_scripts(BEDcoverage_scripts, wkdir)

input file has all mandatory columns! Continue...


In [3]:
gene_regions = intervals_to_dict(gene_intervals_file)

In [4]:
intervals_to_regions(gene_regions)

make_submit_bedscripts_all_genes(gene_regions)